In [117]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import edhec_risk_kit_129 as erk
import pandas as pd
import numpy as np

In [160]:
def show_cppi(n_scenerios=50,mu=0.07,sigma=0.15,m=3,floor=0.0,riskfree_rate=0.03,y_max=100):
    start = 100
    sim_rets = erk.gbm(n_scenarios=n_scenerios,mu=mu, sigma= sigma, prices=False, steps_per_year=12)
    risk_r = pd.DataFrame(sim_rets)
    btr = erk.run_cppi(risky_r=pd.DataFrame(risk_r),riskfree_rate=riskfree_rate,m=m,start=start,floor=floor)
    wealth = btr["Wealth"]
    y_max = wealth.values.max()*y_max/100
    terminal_wealth = wealth.iloc[-1]
    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    failure_mask = np.less(terminal_wealth, start*floor)
    n_failures = failure_mask.sum()
    p_fail = n_failures/n_scenerios
    e_shortfall = np.dot(tv-start*floor,failure_mask)/n_failures if n_failures > 0 else 0.0
    
    fig,(wealth_ax,hist_ax) = plt.subplots(nrows=1,ncols=2,sharey=True,gridspec_kw={'width_ratios':[3,2]},figsize=(24,9))
    plt.subplots_adjust(wspace=0)
    wealth.plot(ax=wealth_ax,legend=False,alpha=0.3,color="red")
    wealth_ax.axhline(y=start,ls=":",color="black")
    wealth_ax.set_ylim(top=y_max)
    wealth_ax.axhline(y=start*floor, ls="--", color="red")
    
    terminal_wealth.plot.hist(ax=hist_ax,bins=50,ec="w", fc="indianred", orientation='horizontal')
    hist_ax.axhline(y=start,ls=":",color="black")
    hist_ax.axhline(y=tw_mean,ls=":",color="blue")
    hist_ax.axhline(y=tw_median,ls=":",color="purple")
    hist_ax.annotate(f"Mean : ${int(tw_mean)}",xy=(0.7,0.9),xycoords='axes fraction', fontsize=24)
    hist_ax.annotate(f"Violations : ${int(n_failures)}",xy=(0.7,0.85),xycoords='axes fraction', fontsize=24)    

In [162]:
cppi_controls = widgets.interactive(show_cppi,n_scenerios=widgets.IntSlider(min=1,max=1000,step=5,value=50),
                                    mu=(0.0,+0.2, 0.01),
                                    sigma=(0,0.3,0.05),
                                    m=(1,5,0.5),
                                    riskfree_rate=(0.0,0.05,0.01),
                                    y_max=widgets.IntSlider(min=1,max=100,step=5,value=50))

In [164]:
display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenerios', max=1000, min=1, step=5), FloatSlider(val…